In [129]:
import sklearn
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

1. Leer el archivo depression.csv

In [184]:
datos = pd.read_csv('depression.csv')

In [185]:
# Variables categóricas (tipo object)
# ==============================================================================
datos.select_dtypes(include=['object']).describe()

,Gender,Sleep Duration,Degree,Suicidal Thoughts,Family History of Mental Illness
count,27901,27901,27901,27901,27901
unique,2,5,5,2,2
top,Male,Less than 5 hours,Undergraduate,Yes,No
freq,15547,8310,12639,17656,14398


In [186]:
# Variables cuantitativas (tipo entero)
# ==============================================================================
datos.select_dtypes(include=['int64']).describe()

,Depression
count,27901.000000
mean,0.585499
std,0.492645
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [187]:
# Variables cuantitativas (tipo float)
# ==============================================================================
datos.select_dtypes(include=['float64']).describe()

,Age,CGPA,Work/Study Hours
count,27901.000000,27901.000000,27901.000000
mean,25.822300,7.656104,7.156984
std,4.905687,1.470707,3.707642
min,18.000000,0.000000,0.000000
25%,21.000000,6.290000,4.000000
50%,25.000000,7.770000,8.000000
75%,30.000000,8.920000,10.000000
max,59.000000,10.000000,12.000000


2. Seleccionar aleatoriamente el 80% del conjunto de datos para entrenar y el 20% restante para las
pruebas

In [188]:
from sklearn.model_selection import train_test_split

N=len(datos)
cTrain=int(N*0.8) # 80% para entrenar y 20% para probar
cTest=N-cTrain
print(N,cTrain,cTest)
train_data,test_data= sklearn.model_selection.train_test_split(datos, train_size=cTrain, test_size=cTest)

27901 22320 5581


In [189]:
train_data.shape

(22320, 9)

In [231]:
train_data.head(5)

,Gender,Age,CGPA,Sleep Duration,Degree,Suicidal Thoughts,Work/Study Hours,Family History of Mental Illness,Depression
17827,Male,31.0,8.74,5-6 hours,Postgraduate,Yes,10.0,No,1
7398,Male,33.0,9.95,More than 8 hours,High School,No,11.0,No,0
9339,Male,25.0,8.50,Less than 5 hours,High School,No,2.0,No,1
16837,Female,24.0,5.77,More than 8 hours,Postgraduate,No,3.0,No,0
11657,Female,31.0,5.76,7-8 hours,Doctorate,Yes,7.0,No,1


3. Utilizar una estrategia para normalizar los datos numéricos y una forma de codificar los atributos
categóricos

## Normalizar los datos numéricos

In [229]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler

num_cols = ["Age", "CGPA", "Work/Study Hours"]
num_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])

### codificar datos categoricos

In [299]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Label Encoding (binarias)
cat_cols_le =  ["Gender", "Suicidal Thoughts", "Family History of Mental Illness"]
cat_pipeline_le = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OrdinalEncoder())  # funciona bien para columnas binarias
])

# One hot enconding
cat_cols_ohe =  ["Sleep Duration", "Degree"]
cat_pipeline_ohe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("cat_encoder", OneHotEncoder(sparse_output=False))
])

### Pipeline completo

In [300]:
from sklearn.compose import ColumnTransformer

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_cols),
    ("cat_ohe", cat_pipeline_ohe, cat_cols_ohe),
    ("cat_le", cat_pipeline_le, cat_cols_le)
])

In [301]:
X_train = full_pipeline.fit_transform(train_data.drop(columns=["Depression"]))

In [302]:
X_train[0,:]

array([1.0542121 , 0.74156593, 0.77224103, 1.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 1.        , 0.        , 1.        , 1.        ,
       0.        ])

In [303]:
y_train = train_data["Depression"]
y_train

17827    1
7398     0
9339     1
16837    0
11657    1
        ..
17090    0
24950    0
25096    1
954      0
7310     0
Name: Depression, Length: 22320, dtype: int64

## Entrenamiento de las redes neuronales

4. Configurar los hiperparámetros del árbol de decisión de la siguiente manera: criterion=gini,
splitter=best, y random_state=123. Obtener 10 árboles de decisión que resultan de modificar el
hiperparámetro max_depth desde 1 hasta 10 con incrementos de 1

In [319]:
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

y_test = test_data["Depression"]
X_test = full_pipeline.fit_transform(test_data.drop(columns=["Depression"]))

results_gini = []
for depth in range(1, 11):
    modelo = tree.DecisionTreeClassifier(
        criterion='gini',
        splitter='best',
        max_depth=depth,
        random_state=123
    )
    modelo.fit(X_train, y_train)
    score = cross_val_score(modelo, X_train, y_train, cv=5,scoring='accuracy')
    y_pred = modelo.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results_gini.append({'max_depth': depth, 'accuracy': score.mean(), 'test accuracy': acc})



print("terminado")

terminado


In [320]:
df_gini = pd.DataFrame(results_gini)
# Accuracy con criterio Gini:
df_gini

,max_depth,accuracy,test accuracy
0,1,0.784543,0.772621
1,2,0.784543,0.772621
2,3,0.784543,0.772621
3,4,0.786783,0.776205
4,5,0.784453,0.776205
5,6,0.784722,0.775847
6,7,0.783199,0.773159
7,8,0.780914,0.773696
8,9,0.774104,0.770471
9,10,0.765860,0.765454
